In [1]:
import os

from urllib.request import urlretrieve
from getpass import getpass

from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pydantic import BaseModel, Field

from typing import List

In [2]:
def split_text(docs: list) -> list:     
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
    return text_splitter.split_documents(docs)

In [3]:
openai_api_key = getpass()

 ········


In [4]:
neo4j_uri = getpass()

 ········


In [5]:
neo4j_username = getpass()

 ········


In [6]:
neo4j_password = getpass()

 ········


In [7]:
model_id = 'gpt-4o'

llm = ChatOpenAI(model=model_id, api_key=openai_api_key)
llm_transformer = LLMGraphTransformer(llm=llm)

In [8]:
register_graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

In [9]:
document_urls = [
    'https://raw.githubusercontent.com/RyoWakabayashi/elixir-learning/main/livebooks/bumblebee/colab/momotaro.txt',
    'https://raw.githubusercontent.com/RyoWakabayashi/elixir-learning/main/livebooks/bumblebee/colab/urashimataro.txt',
    'https://raw.githubusercontent.com/RyoWakabayashi/elixir-learning/main/livebooks/bumblebee/colab/kintaro.txt'
]

for url in document_urls:
    filename = os.path.basename(url)
    urlretrieve(url, filename)

    loader = TextLoader(filename)
    document = loader.load()
    tgt_chunks = split_text(document)

    graph_documents = llm_transformer.convert_to_graph_documents(tgt_chunks)
    register_graph.add_graph_documents(
        graph_documents,
        baseEntityLabel=True,
        include_source=True
    )

In [10]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model='text-embedding-3-small', api_key=openai_api_key),
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [11]:
register_graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id] OPTIONS {indexConfig: {`fulltext.analyzer`: 'cjk'}}"
)

[]

In [12]:
qa_graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

qa_vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model='text-embedding-3-small', api_key=openai_api_key),
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="vector"
)

In [13]:
question = '桃太郎の仲間を教えてください'

In [14]:
unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
unstructured_data

['\ntext: 「うん。」\n\u3000と言いながら、赤さんは抱いているおばあさんの手をはねのけました。\n「おやおや、何という元気のいい子だろう。」\n\u3000おじいさんとおばあさんは、こう言って顔を見合わせながら、「あッは、あッは。」とおもしろそうに笑いました。\n\u3000そして桃の中から生まれた子だというので、この子に桃太郎という名をつけました。',
 '\ntext: 桃太郎もすぐきじの立ったあとから向こうを見ますと、なるほど、遠い遠い海のはてに、ぼんやり雲のような薄ぐろいものが見えました。船の進むにしたがって、雲のように見えていたものが、だんだんはっきりと島の形になって、あらわれてきました。\n「ああ、見える、見える、鬼が島が見える。」\n\u3000桃太郎がこういうと、犬も、猿も、声をそろえて、「万歳、万歳。」とさけびました。\n\u3000見る見る鬼が島が近くなって、もう硬い岩で畳んだ鬼のお城が見えました。いかめしいくろがねの門の前に見はりをしている鬼の兵隊のすがたも見えました。\nそのお城のいちばん高い屋根の上に、きじがとまって、こちらを見ていました。\nこうして何年も、何年もこいで行かなければならないという鬼が島へ、ほんの目をつぶっている間に来たのです。',
 '\ntext: 桃太郎は、犬と猿をしたがえて、船からひらりと陸の上にとび上がりました。\n\u3000見はりをしていた鬼の兵隊は、その見なれないすがたを見ると、びっくりして、あわてて門の中に逃げ込んで、くろがねの門を固くしめてしまいました。その時犬は門の前に立って、\n「日本の桃太郎さんが、お前たちをせいばいにおいでになったのだぞ。あけろ、あけろ。」\n\u3000とどなりながら、ドン、ドン、扉をたたきました。鬼はその声を聞くと、ふるえ上がって、よけい一生懸命に、中から押さえていました。\n\u3000するときじが屋根の上からとび下りてきて、門を押さえている鬼どもの目をつつきまわりましたから、鬼はへいこうして逃げ出しました。その間に、猿がするすると高い岩壁をよじ登っていって、ぞうさなく門を中からあけました。\n「わあッ。」とときの声を上げて、桃太郎の主従が、いさましくお城の中に攻め込んでいきますと、鬼の大将も大ぜいの家来を引き連れて、一人一人、太い鉄の棒をふりまわしながら、「

In [15]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the entities that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

# Entity ExtractionのChain
llm_ner=ChatOpenAI(model_name=model_id, api_key=openai_api_key) 
entity_chain = prompt | llm_ner.with_structured_output(Entities)
entities = entity_chain.invoke({"question": question}).names

entities

['桃太郎']

In [16]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""

    words = [el for el in input.split() if el]
    
    # ANDを末尾につけるのは最後以外
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"

    return full_text_query.strip()

In [17]:
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""

    # Entity Extraction
    entities = entity_chain.invoke({"question": question})

    for entity in entities.names:
        # 方向が逆のものをUNIONしている
        response = qa_graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL (node,node) {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [18]:
structured_retriever(question)

'桃太郎 - DESTINATION -> 鬼が島\n桃太郎 - 持つ -> きびだんご\n桃太郎 - RETURN -> おじいさん\n桃太郎 - RETURN -> おばあさん\n桃太郎 - DEFEATS -> 鬼の大将\n桃太郎 - POSSESSION -> きびだんご\n桃太郎 - COMPANION -> 犬\n桃太郎 - COMPANION -> 猿\n桃太郎 - COMPANION -> きじ\n桃太郎 - SPOKE_TO -> 犬\n桃太郎 - SPOKE_TO -> 猿\n桃太郎 - SPOKE_TO -> きじ\n桃太郎 - 生まれた -> 桃\n桃太郎 - USES -> 船\n桃太郎 - STRONGEST_IN -> 日本\n桃太郎 - INTERESTED_IN -> 鬼が島\n桃太郎 - SEE -> 鬼が島\n桃太郎 - 退治 -> 鬼\n桃太郎 - OWNS -> 日本一のきびだんご\n桃太郎 - LEADS -> 犬\n桃太郎 - LEADS -> 猿\n桃太郎 - CONFLICT -> 鬼の大将\n桃太郎 - TAKE -> 宝物\nおじいさん - RAISED -> 桃太郎\nおばあさん - RAISED -> 桃太郎\nおじいさん - SAID -> 桃太郎\nおばあさん - SAID -> 桃太郎\n犬 - COMPANION -> 桃太郎\n猿 - COMPANION -> 桃太郎\nきじ - COMPANION -> 桃太郎\nおじいさん - 言う -> 桃太郎\nおばあさん - 言う -> 桃太郎\n鬼の大将 - SURRENDER -> 桃太郎\n金太郎 - VISIT -> 都\n金太郎 - DEFEATED -> 熊\n金太郎 - DEFEATS -> 猿\n金太郎 - DEFEATS -> 熊\n金太郎 - DEFEATS -> うさぎ\n金太郎 - DEFEATS -> 鹿\n金太郎 - VISITS -> 森\n金太郎 - ACCOMPANIED_BY -> 貞光\n金太郎 - GOES_TO -> 森\n金太郎 - LIVES_WITH -> 山うば\n金太郎 - CALLS -> 猿\n金太郎 - CALLS -> 熊\n金太郎 - CALLS -> うさぎ\n金太郎 - CALLS -> 鹿

In [20]:
def retriever(question: str):
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(final_data)
    return final_data

retriever(question)

Structured data:
桃太郎 - DESTINATION -> 鬼が島
桃太郎 - 持つ -> きびだんご
桃太郎 - RETURN -> おじいさん
桃太郎 - RETURN -> おばあさん
桃太郎 - DEFEATS -> 鬼の大将
桃太郎 - POSSESSION -> きびだんご
桃太郎 - COMPANION -> 犬
桃太郎 - COMPANION -> 猿
桃太郎 - COMPANION -> きじ
桃太郎 - SPOKE_TO -> 犬
桃太郎 - SPOKE_TO -> 猿
桃太郎 - SPOKE_TO -> きじ
桃太郎 - 生まれた -> 桃
桃太郎 - USES -> 船
桃太郎 - STRONGEST_IN -> 日本
桃太郎 - INTERESTED_IN -> 鬼が島
桃太郎 - SEE -> 鬼が島
桃太郎 - 退治 -> 鬼
桃太郎 - OWNS -> 日本一のきびだんご
桃太郎 - LEADS -> 犬
桃太郎 - LEADS -> 猿
桃太郎 - CONFLICT -> 鬼の大将
桃太郎 - TAKE -> 宝物
おじいさん - RAISED -> 桃太郎
おばあさん - RAISED -> 桃太郎
おじいさん - SAID -> 桃太郎
おばあさん - SAID -> 桃太郎
犬 - COMPANION -> 桃太郎
猿 - COMPANION -> 桃太郎
きじ - COMPANION -> 桃太郎
おじいさん - 言う -> 桃太郎
おばあさん - 言う -> 桃太郎
鬼の大将 - SURRENDER -> 桃太郎
金太郎 - VISIT -> 都
金太郎 - DEFEATED -> 熊
金太郎 - DEFEATS -> 猿
金太郎 - DEFEATS -> 熊
金太郎 - DEFEATS -> うさぎ
金太郎 - DEFEATS -> 鹿
金太郎 - VISITS -> 森
金太郎 - ACCOMPANIED_BY -> 貞光
金太郎 - GOES_TO -> 森
金太郎 - LIVES_WITH -> 山うば
金太郎 - CALLS -> 猿
金太郎 - CALLS -> 熊
金太郎 - CALLS -> うさぎ
金太郎 - CALLS -> 鹿
金太郎 - CHILD_OF -> おかあさん
金太郎 -

'Structured data:\n桃太郎 - DESTINATION -> 鬼が島\n桃太郎 - 持つ -> きびだんご\n桃太郎 - RETURN -> おじいさん\n桃太郎 - RETURN -> おばあさん\n桃太郎 - DEFEATS -> 鬼の大将\n桃太郎 - POSSESSION -> きびだんご\n桃太郎 - COMPANION -> 犬\n桃太郎 - COMPANION -> 猿\n桃太郎 - COMPANION -> きじ\n桃太郎 - SPOKE_TO -> 犬\n桃太郎 - SPOKE_TO -> 猿\n桃太郎 - SPOKE_TO -> きじ\n桃太郎 - 生まれた -> 桃\n桃太郎 - USES -> 船\n桃太郎 - STRONGEST_IN -> 日本\n桃太郎 - INTERESTED_IN -> 鬼が島\n桃太郎 - SEE -> 鬼が島\n桃太郎 - 退治 -> 鬼\n桃太郎 - OWNS -> 日本一のきびだんご\n桃太郎 - LEADS -> 犬\n桃太郎 - LEADS -> 猿\n桃太郎 - CONFLICT -> 鬼の大将\n桃太郎 - TAKE -> 宝物\nおじいさん - RAISED -> 桃太郎\nおばあさん - RAISED -> 桃太郎\nおじいさん - SAID -> 桃太郎\nおばあさん - SAID -> 桃太郎\n犬 - COMPANION -> 桃太郎\n猿 - COMPANION -> 桃太郎\nきじ - COMPANION -> 桃太郎\nおじいさん - 言う -> 桃太郎\nおばあさん - 言う -> 桃太郎\n鬼の大将 - SURRENDER -> 桃太郎\n金太郎 - VISIT -> 都\n金太郎 - DEFEATED -> 熊\n金太郎 - DEFEATS -> 猿\n金太郎 - DEFEATS -> 熊\n金太郎 - DEFEATS -> うさぎ\n金太郎 - DEFEATS -> 鹿\n金太郎 - VISITS -> 森\n金太郎 - ACCOMPANIED_BY -> 貞光\n金太郎 - GOES_TO -> 森\n金太郎 - LIVES_WITH -> 山うば\n金太郎 - CALLS -> 猿\n金太郎 - CALLS -> 熊\n金太郎 - CALLS -> うさぎ

In [21]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | ChatOpenAI(temperature=0, model_name=model_id, api_key=openai_api_key)
    | StrOutputParser()
)

In [22]:
chain.invoke(question)

Structured data:
桃太郎 - DESTINATION -> 鬼が島
桃太郎 - 持つ -> きびだんご
桃太郎 - RETURN -> おじいさん
桃太郎 - RETURN -> おばあさん
桃太郎 - DEFEATS -> 鬼の大将
桃太郎 - POSSESSION -> きびだんご
桃太郎 - COMPANION -> 犬
桃太郎 - COMPANION -> 猿
桃太郎 - COMPANION -> きじ
桃太郎 - SPOKE_TO -> 犬
桃太郎 - SPOKE_TO -> 猿
桃太郎 - SPOKE_TO -> きじ
桃太郎 - 生まれた -> 桃
桃太郎 - USES -> 船
桃太郎 - STRONGEST_IN -> 日本
桃太郎 - INTERESTED_IN -> 鬼が島
桃太郎 - SEE -> 鬼が島
桃太郎 - 退治 -> 鬼
桃太郎 - OWNS -> 日本一のきびだんご
桃太郎 - LEADS -> 犬
桃太郎 - LEADS -> 猿
桃太郎 - CONFLICT -> 鬼の大将
桃太郎 - TAKE -> 宝物
おじいさん - RAISED -> 桃太郎
おばあさん - RAISED -> 桃太郎
おじいさん - SAID -> 桃太郎
おばあさん - SAID -> 桃太郎
犬 - COMPANION -> 桃太郎
猿 - COMPANION -> 桃太郎
きじ - COMPANION -> 桃太郎
おじいさん - 言う -> 桃太郎
おばあさん - 言う -> 桃太郎
鬼の大将 - SURRENDER -> 桃太郎
金太郎 - VISIT -> 都
金太郎 - DEFEATED -> 熊
金太郎 - DEFEATS -> 猿
金太郎 - DEFEATS -> 熊
金太郎 - DEFEATS -> うさぎ
金太郎 - DEFEATS -> 鹿
金太郎 - VISITS -> 森
金太郎 - ACCOMPANIED_BY -> 貞光
金太郎 - GOES_TO -> 森
金太郎 - LIVES_WITH -> 山うば
金太郎 - CALLS -> 猿
金太郎 - CALLS -> 熊
金太郎 - CALLS -> うさぎ
金太郎 - CALLS -> 鹿
金太郎 - CHILD_OF -> おかあさん
金太郎 -

'桃太郎の仲間は、犬、猿、きじです。'

In [23]:
chain.invoke('浦島太郎を乗せたのは誰？')

Structured data:
浦島太郎 - LIVES_IN -> 丹後の国水の江の浦
浦島太郎 - HAS -> 玉手箱
浦島太郎 - VISITED -> りゅう宮
浦島太郎 - SAVES -> かめの子
浦島太郎 - FEEDS -> 浦島太郎の父
浦島太郎 - FEEDS -> 浦島太郎の母
おばあさん - MENTIONED -> 浦島太郎
浦島 - 話しました -> 浦島太郎
おばあさん - 知らない -> 浦島太郎
桃太郎 - DESTINATION -> 鬼が島
桃太郎 - 持つ -> きびだんご
桃太郎 - RETURN -> おじいさん
桃太郎 - RETURN -> おばあさん
桃太郎 - DEFEATS -> 鬼の大将
桃太郎 - POSSESSION -> きびだんご
桃太郎 - COMPANION -> 犬
桃太郎 - COMPANION -> 猿
桃太郎 - COMPANION -> きじ
桃太郎 - SPOKE_TO -> 犬
桃太郎 - SPOKE_TO -> 猿
桃太郎 - SPOKE_TO -> きじ
桃太郎 - 生まれた -> 桃
桃太郎 - USES -> 船
桃太郎 - STRONGEST_IN -> 日本
桃太郎 - INTERESTED_IN -> 鬼が島
桃太郎 - SEE -> 鬼が島
桃太郎 - 退治 -> 鬼
桃太郎 - OWNS -> 日本一のきびだんご
桃太郎 - LEADS -> 犬
桃太郎 - LEADS -> 猿
桃太郎 - CONFLICT -> 鬼の大将
桃太郎 - TAKE -> 宝物
おじいさん - RAISED -> 桃太郎
おばあさん - RAISED -> 桃太郎
おじいさん - SAID -> 桃太郎
おばあさん - SAID -> 桃太郎
犬 - COMPANION -> 桃太郎
猿 - COMPANION -> 桃太郎
きじ - COMPANION -> 桃太郎
おじいさん - 言う -> 桃太郎
おばあさん - 言う -> 桃太郎
鬼の大将 - SURRENDER -> 桃太郎
Unstructured data:

text: 「桃太郎さん、桃太郎さん、どちらへおいでになります。」
　とたずねました。
「鬼が島へ鬼せいばつに行くのだ。」
「お腰に下げたものは、何でございま

'浦島太郎を乗せたのは、助けたかめです。'

In [25]:
chain.invoke('金太郎の家来は誰？')

Structured data:
金太郎 - VISIT -> 都
金太郎 - DEFEATED -> 熊
金太郎 - DEFEATS -> 猿
金太郎 - DEFEATS -> 熊
金太郎 - DEFEATS -> うさぎ
金太郎 - DEFEATS -> 鹿
金太郎 - VISITS -> 森
金太郎 - ACCOMPANIED_BY -> 貞光
金太郎 - GOES_TO -> 森
金太郎 - LIVES_WITH -> 山うば
金太郎 - CALLS -> 猿
金太郎 - CALLS -> 熊
金太郎 - CALLS -> うさぎ
金太郎 - CALLS -> 鹿
金太郎 - CHILD_OF -> おかあさん
金太郎 - INTRODUCED_TO -> 頼光
金太郎 - RENAMED_TO -> 坂田金時
金太郎 - ARRIVED_AT -> 谷川
金太郎 - 話 -> 山うば
金太郎 - BIRTHPLACE -> 相模国足柄山
金太郎 - RECEIVES_FOOD_FROM -> おかあさん
金太郎 - OFFERS_AS_REWARD -> おむすび
金太郎 - PUSHED_DOWN -> 杉の木
金太郎 - ENTERED -> 山奥の一軒家
金太郎 - 相撲 -> おじさん
金太郎 - 倒す -> 杉の木
金太郎 - 可能性 -> 勇士
金太郎 - 願望 -> お侍
金太郎 - 志望 -> 都
猿 - SERVANT_OF -> 金太郎
熊 - SERVANT_OF -> 金太郎
うさぎ - SERVANT_OF -> 金太郎
鹿 - SERVANT_OF -> 金太郎
碓井貞光 - PROPOSED -> 金太郎
きこり - 提案 -> 金太郎
きこり - OBSERVED -> 金太郎
金太郎の父 - DESCENDANT_OF -> 坂田
山うば - SPOUSE -> 金太郎の父
Unstructured data:

text: むかし、金太郎という強い子供がありました。相模国足柄山の山奥に生まれて、おかあさんの山うばといっしょにくらしていました。
　金太郎は生まれた時からそれはそれは力が強くって、もう七つ八つのころには、石臼やもみぬかの俵ぐらい、へいきで持ち上げました。大抵の大人を相手にすもうを取っても負けませんでした。近所

'金太郎の家来は、熊、猿、うさぎ、鹿です。'